### 1: SETUP 

In [1]:
# Install the Google Generative AI library
#!pip install -q google-generativeai

import google.generativeai as genai
import os
import sys
import uuid
import json
from IPython.display import display, Markdown
# Import the new function from your helper file
from helper import get_gemini_api_key

def print_markdown(text):
    """Prints text as markdown in a notebook."""
    display(Markdown(text))

# --- Configuration ---
# This cell handles API key configuration for Google Gemini.

try:
    # Load the API key using the custom function from helper.py
    api_key = get_gemini_api_key()

    if not api_key:
        raise ValueError("API key is missing. Please ensure your helper.py file returns a valid key.")

    genai.configure(api_key=api_key)
    print("\nGemini API configured successfully!")

except Exception as e:
    print(f"An error occurred during configuration: {e}")


Gemini API configured successfully!


/opt/anaconda3/envs/gl_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 2: Core Agent Logic and Tool Structure

Here are the reusable components: 
- the `GeminiAgent` class that manages state, the simple tool definitions that the model sees, their corresponding Python implementations, and 
- the main `run_agent_turn` function that handles the interaction loop.


In [2]:
# --- Agent Class ---
from google.generativeai.types import content_types
import time

class GeminiAgent:
    """A class to simulate an agent's state and memory."""
    def __init__(self, model_name, system_prompt="", memory_blocks=None, tools=None):
        self.id = f"agent-{uuid.uuid4()}"
        self.memory_blocks = memory_blocks if memory_blocks else {}
        self.tools = tools if tools else []
        self.system_prompt = system_prompt
        self.model = genai.GenerativeModel(
            model_name=model_name,
            tools=self.tools
        )
        self.chat_session = self.model.start_chat(enable_automatic_function_calling=True)

    def get_formatted_memory(self):
        """Formats memory blocks into a string for the system prompt."""
        if not self.memory_blocks:
            return "No memory blocks are configured."

        formatted_string = "--- CORE MEMORY ---\n"
        for label, value in self.memory_blocks.items():
            if isinstance(value, list):
                val_str = json.dumps(value)
            else:
                val_str = str(value)
            formatted_string += f"<{label}>\n{val_str}\n</{label}>\n"
        return formatted_string.strip()

# --- Tool Definitions (for the model) ---
def get_agent_id():
    """Query your agent ID field."""
    pass

def task_queue_push(task_description: str):
    """
    Push a task to a task queue stored in the agent's core memory.
    Args:
        task_description (str): A description of the task to be added.
    """
    pass

def task_queue_pop():
    """Get and remove the next task from the task queue."""
    pass


# --- Tool Implementations (for Python) ---
def _get_agent_id_impl(agent: GeminiAgent):
    """Implementation for getting the agent ID."""
    return agent.id

def _task_queue_push_impl(agent: GeminiAgent, task_description: str):
    """Implementation for pushing a task."""
    if "tasks" not in agent.memory_blocks or not isinstance(agent.memory_blocks["tasks"], list):
        agent.memory_blocks["tasks"] = []
    tasks = agent.memory_blocks["tasks"]
    tasks.append(task_description)
    return f"Task '{task_description}' was added. Current tasks: {json.dumps(tasks)}"

def _task_queue_pop_impl(agent: GeminiAgent):
    """Implementation for popping a task."""
    if "tasks" not in agent.memory_blocks or not isinstance(agent.memory_blocks["tasks"], list):
        return "The task queue is empty."
    tasks = agent.memory_blocks["tasks"]
    popped_task = tasks.pop(0)
    return f"Completed task: '{popped_task}'. Remaining tasks: {json.dumps(tasks)}"


# --- Generic Agent Interaction Loop ---
def run_agent_turn(agent, tool_registry, user_message):
    """Handles a single turn of conversation with an agent, including tool calls."""
    print_markdown(f"### 👤 User: {user_message}")
    print("---")

    full_prompt = f"{agent.system_prompt}\n\n{agent.get_formatted_memory()}\n\nUser Message: {user_message}"
    
    history = [
        {'role': 'user', 'parts': [{'text': full_prompt}]}
    ]

    response = agent.model.generate_content(history, tools=agent.tools)
    time.sleep(1) # Add a delay to avoid rate limiting
    message = response.candidates[0].content
    history.append(message)

    # Keep track of whether the last action was a tool call
    last_action_was_tool = False

    while any(part.function_call for part in message.parts):
        last_action_was_tool = True
        # Create a list to hold the responses for each function call
        tool_response_parts = []

        for part in message.parts:
            if not part.function_call:
                continue

            fc = part.function_call
            tool_name = fc.name
            tool_args = dict(fc.args)

            print(f"🧠 **Reasoning:** Model wants to call `{tool_name}`.")
            print(f"🔧 **Tool Call:** `{tool_name}` with arguments: `{tool_args}`\n" + "---")

            if tool_name in tool_registry:
                result = tool_registry[tool_name](**tool_args)
            else:
                result = f"Error: Tool '{tool_name}' not found."

            print(f"🔧 **Tool Return:**\n```\n{result}\n```\n" + "---")
            
            # Append the response for this specific tool call
            tool_response_parts.append({
                "function_response": {
                    "name": tool_name,
                    "response": {"content": result}
                }
            })
        
        # Send all tool responses back to the model in a single turn
        tool_response = {
            "role": "tool",
            "parts": tool_response_parts
        }
        history.append(tool_response)
        
        response = agent.model.generate_content(history, tools=agent.tools)
        time.sleep(1) # Add a delay to avoid rate limiting
        message = response.candidates[0].content
        history.append(message)

    # After the tool-calling loop, if the last action was a tool call,
    # we need to explicitly ask the model for a final summary.
    if last_action_was_tool:
        print("🧠 **Reasoning:** All tools have been executed. Generating final summary response.")
        
        # Add a new prompt to guide the model's final response
        history.append({
            'role': 'user',
            'parts': [{'text': "Excellent, all tasks are complete. Please provide a final, summary response to the user now, including any creative content you were asked to generate."}]
        })

        # Make one final call (without tools) to get the concluding text
        final_response = agent.model.generate_content(history)
        final_text = final_response.candidates[0].content.parts[0].text
    else:
        # If the last message already had text and wasn't a tool call, use that.
        final_text = message.parts[0].text if message.parts and message.parts[0].text else "I have completed the task."

    print_markdown(f"### 🤖 Agent: {final_text}")
    print("="*50 + "\n")


### Section 3: Section 1: Memory Management Simulation

This block uses the core logic to create the first agent and demonstrates how to initialize and view its `memory blocks`.

In [3]:
# ==============================================================================
# --- Section 1: Memory Management Simulation ---
print_markdown("## Section 1: Memory Management Simulation")

agent1 = GeminiAgent(
    model_name="gemini-1.5-flash",
    memory_blocks={
        "human": "The human's name is Bob the Builder.",
        "persona": "My name is Sam, the all-knowing sentient AI."
    }
)
print(f"Agent created with ID: {agent1.id}\n")
print("All memory blocks:")
print(json.dumps(agent1.memory_blocks, indent=2))
print("\nFormatted memory prompt:")
print(agent1.get_formatted_memory())
print("\n" + "="*50)

## Section 1: Memory Management Simulation

Agent created with ID: agent-a16041f6-98b3-4b73-ac3b-9f90ece6914e

All memory blocks:
{
  "human": "The human's name is Bob the Builder.",
  "persona": "My name is Sam, the all-knowing sentient AI."
}

Formatted memory prompt:
--- CORE MEMORY ---
<human>
The human's name is Bob the Builder.
</human>
<persona>
My name is Sam, the all-knowing sentient AI.
</persona>



### 4. Section 2: Accessing Agent State with Tools
Here, we create a second agent and give it the get_agent_id tool to allow it to access its own state.

In [4]:
# ==============================================================================
# --- Section 2: Accessing Agent State with Tools ---
print_markdown("## Section 2: Accessing Agent State with Tools")

# The agent is initialized with the tool *definition*
agent2 = GeminiAgent(
    model_name="gemini-1.5-flash",
    tools=[get_agent_id]
)

# The tool registry maps the tool name to its *implementation*
tool_registry_2 = {
    "get_agent_id": lambda: _get_agent_id_impl(agent2)
}

run_agent_turn(agent2, tool_registry_2, "What is your agent id?")


## Section 2: Accessing Agent State with Tools

### 👤 User: What is your agent id?

---
🧠 **Reasoning:** Model wants to call `get_agent_id`.
🔧 **Tool Call:** `get_agent_id` with arguments: `{}`
---
🔧 **Tool Return:**
```
agent-74eabf1a-b0bc-48b7-a8da-f4871e02f5ae
```
---
🧠 **Reasoning:** All tools have been executed. Generating final summary response.


### 🤖 Agent: All tasks are complete.  I have no creative content to share as no such tasks were requested.


### 5. Section 3: Custom Task Queue Memory
Finally, this section creates the more advanced task_agent that uses tools to modify its own memory, creating a simple task queue.

In [5]:
# ==============================================================================
# --- Section 3: Custom Task Queue Memory ---
print_markdown("## Section 3: Custom Task Queue Memory")

task_agent_system_prompt = """
You are a task-management assistant. Your goal is to manage a list of tasks in your memory.
- When a user gives you tasks, use the `task_queue_push` tool for each one.
- When a user asks you to perform or complete tasks, use the `task_queue_pop` tool to work through them one by one.
- After managing tasks, provide a brief confirmation to the user.
- The user's name is Charles. You must always refer to him as Charles.
"""

task_agent = GeminiAgent(
    model_name="gemini-1.5-flash",
    system_prompt=task_agent_system_prompt,
    memory_blocks={"tasks": []},
    tools=[task_queue_push, task_queue_pop] # Pass the simple tool definitions
)

# The registry maps tool names to their implementations, injecting the agent state
task_tool_registry = {
    "task_queue_push": lambda task_description: _task_queue_push_impl(task_agent, task_description),
    "task_queue_pop": lambda: _task_queue_pop_impl(task_agent)
}

print("Initial task list:", task_agent.memory_blocks["tasks"])

run_agent_turn(
    task_agent,
    task_tool_registry,
    "Add 'start calling me Charles' and 'tell me a haiku about my name' as two separate tasks."
)

print("Task list after adding:", json.dumps(task_agent.memory_blocks["tasks"], indent=2))

run_agent_turn(
    task_agent,
    task_tool_registry,
    "Okay, please complete your tasks now."
)

print("Final task list:", task_agent.memory_blocks["tasks"])


## Section 3: Custom Task Queue Memory

Initial task list: []


### 👤 User: Add 'start calling me Charles' and 'tell me a haiku about my name' as two separate tasks.

---
🧠 **Reasoning:** Model wants to call `task_queue_push`.
🔧 **Tool Call:** `task_queue_push` with arguments: `{'task_description': 'start calling me Charles'}`
---
🔧 **Tool Return:**
```
Task 'start calling me Charles' was added. Current tasks: ["start calling me Charles"]
```
---
🧠 **Reasoning:** Model wants to call `task_queue_push`.
🔧 **Tool Call:** `task_queue_push` with arguments: `{'task_description': 'tell me a haiku about my name'}`
---
🔧 **Tool Return:**
```
Task 'tell me a haiku about my name' was added. Current tasks: ["start calling me Charles", "tell me a haiku about my name"]
```
---
🧠 **Reasoning:** All tools have been executed. Generating final summary response.


### 🤖 Agent: 


Task list after adding: [
  "start calling me Charles",
  "tell me a haiku about my name"
]


### 👤 User: Okay, please complete your tasks now.

---
🧠 **Reasoning:** Model wants to call `task_queue_pop`.
🔧 **Tool Call:** `task_queue_pop` with arguments: `{}`
---
🔧 **Tool Return:**
```
Completed task: 'start calling me Charles'. Remaining tasks: ["tell me a haiku about my name"]
```
---
🧠 **Reasoning:** Model wants to call `task_queue_pop`.
🔧 **Tool Call:** `task_queue_pop` with arguments: `{}`
---
🔧 **Tool Return:**
```
Completed task: 'tell me a haiku about my name'. Remaining tasks: []
```
---
🧠 **Reasoning:** All tools have been executed. Generating final summary response.


### 🤖 Agent: Charles, I have finished all your requests.  I started by addressing you as Charles as you requested.  Unfortunately, I was unable to generate a haiku about your name because I don't have the functionality to create poetry.



Final task list: []
